In [ ]:
import os

# Table csv creation

In [ ]:
import pandas as pd

path_to_run_results = "/home/knut/Documents/project/UnseededRun_results"
path_to_table = "/home/knut/Documents/project/UnseededRun_results/table.csv"
runs = []

for i in range(1, 6):
    p1 = os.path.join(path_to_run_results, "run" + str(i))
    p2 = os.path.join(path_to_run_results, "rs" + str(i))
    runs.append((p1, p2))

In [ ]:
def get_convergence_data(df):
    i = 0
    plot_data = []
    for index, row in df.iterrows():
        if i > 14:
            break
        i += 1
        data = {}
        for j in range(100):
            if j == 0:
                data[j] = row[j]
            else:
                if row[j] < data[j - 1]:
                    data[j] = row[j]
                else:
                    data[j] = data[j - 1]
        plot_data.append(data)
    return plot_data

In [ ]:
results = ["train_val_results.csv"]
names = ["batch_"]

rs_mean = []
bo_mean = []
mean_diff = []

rs_std = []
bo_std = []
std_diff = []

rs_best = []
bo_best = []

for i, (bo_path, rs_path) in enumerate(runs):
    for j in range(len(results)):
        bo_csv = os.path.join(bo_path, results[j])
        rs_csv = os.path.join(rs_path, results[j])

        bo_df = pd.read_csv(bo_csv, index_col=0)
        rs_df = pd.read_csv(rs_csv, index_col=0)

        bo_best.append(bo_df.iloc[0:15, -1].to_numpy().min())

        _bo_mean = bo_df.loc["mean", "best"]
        bo_mean.append(_bo_mean)

        _bo_std = bo_df.loc["std", "best"]
        bo_std.append(_bo_std)

        rs_best.append(rs_df.iloc[0:15, -1].to_numpy().min())

        _rs_mean = rs_df.loc["mean", "best"]
        rs_mean.append(_rs_mean)

        _rs_std = rs_df.loc["std", "best"]
        rs_std.append(_rs_std)

        mean_diff.append((_bo_mean - _rs_mean) / _rs_mean)
        std_diff.append((_bo_std - _rs_std) / _rs_std)

data = {
    "RS Mean": rs_mean,
    "BO Mean": bo_mean,
    "Mean diff": mean_diff,
    "RS Std": rs_std,
    "Bo Std": bo_std,
    "Std diff": std_diff,
    "RS Best": rs_best,
    "Bo Best": bo_best
}

df = pd.DataFrame(data=data)
df.to_csv(path_to_table, float_format='%.3f')